In [1]:
from google import genai
from google.genai import types
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    api_key=os.environ['GEMINI_API_KEY'],
    base_url="https://generativelanguage.googleapis.com/v1beta/"
)

def call_gemini(system_message, user_message, response_schema = None, model="gemini-2.5-flash-lite"):
    if response_schema:
        response = client.chat.completions.parse(
            model = model,
            messages = [
                {
                    "role": "system",
                    "content": system_message
                },
                {
                    "role": "user",
                    "content": user_message
                }
            ],
            temperature = 0.65,
            extra_body={
                
            },
            response_format = response_schema if response_schema else None
        ).choices[0].message.parsed    
        return response
    else:
        response = client.chat.completions.create(
            model = model,
            messages = [
                {
                    "role": "system",
                    "content": system_message
                },
                {
                    "role": "user",
                    "content": user_message
                }
            ],
            temperature = 0.65,
            extra_body={
                
            }
        ).choices[0].message.content    
        return response

In [16]:
import json
import numpy as np
with open('similar_question_data.json','r') as f:
    dataset = json.load(f)

In [8]:
dataset = np.asarray(dataset)

In [31]:
np.random.choice(dataset,4,replace=False).tolist()

[{'question_id': '68a2e',
  'question_text': 'Find the number of 7 lettered palindromes which can be formed using the letters from the English alphabets.',
  'subject': 'Mathematics',
  'similar_questions': [{'similar_question_text': '60 words can be made using all the letters of the word $$\\mathrm{BHBJO}$$, with or without meaning. If these words are written as in a dictionary, then the $$50^{\\text {th }}$$ word is:',
    'similarity_score': 0.9665,
    'summarized_solution_approach': "To find the 50th word in the dictionary order of permutations of the letters BHBJO, we first calculate the total permutations, accounting for the repeated 'B'. We then systematically count the words starting with each letter in alphabetical order (B, H, J, O) to determine which starting letter contains the 50th word. After finding that the 50th word starts with 'O', we find the first two permutations starting with 'O' to identify the 50th word."},
   {'similar_question_text': "The number of 4 letter w

In [ ]:
from prompts import 
from datatypes import RelevanceSimilarity, RelevanceAlignment, EvaluationReport

In [4]:
ques1 = dataset[325]
print(ques1['question_text'])
print(ques1['similar_questions'])

माना S = {√n: 1≤ n ≤ 50 तथा n विषम है} माना a ∈ S तथा A = [[1, 0, a], [-1, 1, 0], [-a, 0, 1]] हैं। यदि ∑ det (adj A) = 100λ है, तो λ बराबर है :

The image contains a mathematical problem involving a set S, a matrix A, and a summation. The set S is defined as the set of square roots of odd integers n, where n ranges from 1 to 50. A 3x3 matrix A is given, with elements that include a variable 'a' which belongs to the set S. The problem involves calculating the sum of the determinants of the adjugate of matrix A, where 'a' takes on values from the set S. This sum is equated to 100λ, and the task is to find the value of λ.
[{'similar_question_text': "If \\(D_{r}=\\begin{vmatrix}r & x & \\frac{n(n+1)}{2} \\\\ 2r-1 & y & n^{2} \\\\ 3r-2 & z & \\frac{n(3n-1)}{2}\\end{vmatrix}\\), then \\(\\sum_{r=1}^{n} D_{r}\\) is equal to :\n\nThe image presents a mathematical problem involving a 3x3 determinant denoted as D_r, where the elements of the determinant are expressions involving 'r' and 'n'. The

In [ ]:
formatted_similar_questions = "\n\n".join([f"### QUESTION {idx + 1}\n- {sq['similar_question_text']}" for idx,sq in enumerate(ques1['similar_questions'])])
formatted_system_prompt = relevance_similarity_system_prompt.format(subject = ques1['subject'])
formatted_user_prompt = relevance_user_prompt.format(main_question = ques1['question_text'], similar_questions = formatted_similar_questions)
print(call_gemini(formatted_system_prompt, formatted_user_prompt, RelevanceSimilarity, model="gemini-2.5-flash"))

conceptual_similarity=1.0 structural_similarity=1.0 reasoning='This question is an exact duplicate of the main question. Both questions define the identical set S = {√n: 1≤ n ≤ 50 तथा n विषम है}, the identical matrix A = [[1, 0, a], [-1, 1, 0], [-a, 0, 1]], and ask for the identical summation ∑ det (adj A) = 100λ. Consequently, they test precisely the same underlying concepts, including set definition, matrix determinant calculation, properties of adjugate matrices (det(adj A) = (det A)^(n-1)), and summation over a specified set. Furthermore, they share the exact same problem structure, setup, and logical/mathematical framework, making them completely analogous.'


In [6]:
formatted_similar_questions = "\n\n".join([f"### QUESTION {idx + 1}\n- {sq['similar_question_text']}\n### SOLUTION APPROACH {idx + 1}\n- {sq['summarized_solution_approach']}" for idx,sq in enumerate(ques1['similar_questions'])])
formatted_system_prompt = relevance_alignment_system_prompt.format(subject = ques1['subject'])
formatted_user_prompt = relevance_user_prompt.format(main_question = ques1['question_text'], similar_questions = formatted_similar_questions)
print(call_gemini(formatted_system_prompt, formatted_user_prompt, EvaluationReport, model="gemini-2.5-flash"))

is_difficulty_appropriate=<AppropriateAlignment.YES: 'YES'> is_solution_approach_viable=<AppropriateAlignment.YES: 'YES'> reasoning="For difficulty, both the main question and similar questions involve concepts such as set definition, matrix operations (determinants, adjugate matrix properties), and summation of sequences. These are standard topics taught in high school advanced mathematics or early undergraduate linear algebra (e.g., JEE Advanced level curriculum). They do not require any PhD-level knowledge. Therefore, the difficulty is appropriate for students.For solution approach viability, Solution Approach 2 is perfectly viable as Similar Question 2 is an exact duplicate of the Main Question. This approach correctly identifies and applies the key property det(adj A) = (det A)^(n-1) (where n is the order of the matrix), calculates the determinant of A, correctly identifies the elements of the set S, and outlines the required summation. This method directly solves the Main Questio